# Phase 6: Real Quantum Hardware & NISQ Computing - Core

**Author**: Wadoud Charbak  
**Date**: November 2024  
**For**: Quantinuum & Riverlane Recruitment

---

## Overview

Welcome to Phase 6, the final phase of this quantum computing journey! This notebook bridges the gap between theory (Phases 1-5) and practice by exploring **real quantum hardware** and **NISQ (Noisy Intermediate-Scale Quantum)** computing.

### What You'll Learn

1. **Hardware Platform Comparison** - IBM, IonQ, Rigetti specifications
2. **Realistic Noise Modeling** - Hardware-specific noise characteristics
3. **Running Algorithms with Noise** - See theory meet reality
4. **Error Mitigation Techniques** - ZNE, PEC, readout calibration

### Prerequisites

- Completed Phases 1-5 (or understand qubits, entanglement, algorithms, noise, error correction)
- Python 3.8+
- Libraries: numpy, matplotlib, qiskit (optional for visualization)

Let's begin!

In [ ]:
# Imports
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List

# Add src to path
sys.path.append(os.path.abspath('../src'))

# Import Phase 6 modules
from phase6_hardware import (
    get_backend_specs,
    compare_backends,
    create_noise_model,
    plot_backend_comparison,
    plot_circuit_fidelity_vs_depth,
    plot_error_mitigation_comparison,
    plot_decoherence_curves,
    ReadoutErrorMitigator,
    ZeroNoiseExtrapolation
)

# Configure plotting
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✓ All imports successful!")
print("\nPhase 6: Real Quantum Hardware & NISQ Computing")
print("="*70)

---

## Part 1: Quantum Hardware Platform Comparison

### The NISQ Era

We are currently in the **Noisy Intermediate-Scale Quantum (NISQ)** era:
- **50-1000 qubits** (intermediate scale)
- **High error rates** (~0.1-1% per gate)
- **No error correction** (not enough qubits yet)
- **Shallow circuits** (limited by decoherence)

Let's explore the three major platforms available today.

### 1.1 Hardware Specifications

Each platform uses different physical implementations with distinct trade-offs.

In [ ]:
# Get specifications for each platform
backends = ['ibm_jakarta', 'ionq_harmony', 'rigetti_aspen_m3']

print("QUANTUM HARDWARE SPECIFICATIONS")
print("="*70)

for backend_name in backends:
    specs = get_backend_specs(backend_name)
    
    print(f"\n{backend_name.upper()}")
    print("-"*70)
    print(f"Technology:      {specs.backend_type.value}")
    print(f"Qubits:          {specs.num_qubits}")
    print(f"Connectivity:    {specs.connectivity_degree():.1%} (fraction of max)")
    print(f"Average T1:      {np.mean(specs.t1_times):.2f} μs")
    print(f"Average T2:      {np.mean(specs.t2_times):.2f} μs")
    print(f"T2/T1 ratio:     {np.mean(specs.t2_times)/np.mean(specs.t1_times):.3f}")
    print(f"Single-Q fidelity: {specs.gate_fidelities.get('single_qubit', 1.0):.6f}")
    print(f"Two-Q fidelity:    {specs.gate_fidelities.get('cnot', 1.0):.6f}")
    print(f"Readout fidelity:  {specs.readout_fidelity:.6f}")

print("\n" + "="*70)

### 1.2 Key Observations

**IBM (Superconducting Qubits)**:
- T₁ ~100-150 μs (moderate coherence)
- Limited connectivity (nearest-neighbor)
- Very good gate fidelities (99.5%+)
- Fast gate times (~50ns for single-qubit)

**IonQ (Trapped Ions)**:
- T₁ ~1,000,000 μs = **1000 seconds!** (essentially infinite)
- All-to-all connectivity (any qubit can interact with any other)
- Excellent single-qubit fidelity (99.98%)
- Slower two-qubit gates (~500μs)

**Rigetti (Superconducting Qubits)**:
- T₁ ~15-30 μs (shorter coherence)
- Limited connectivity (line topology)
- Lower two-qubit fidelities (~90%)
- Many qubits (80+)

**Key Insight**: No perfect platform! Trade-offs between:
- Coherence time vs gate speed
- Fidelity vs number of qubits
- Connectivity vs error rates

In [ ]:
# Comprehensive visual comparison
fig = plot_backend_comparison(backends)
plt.show()

print("\n💡 Notice:")
print("   • IonQ has ~10,000x longer T1 than others!")
print("   • IBM has best balance of qubits and fidelity")
print("   • Rigetti has most qubits but lower fidelities")
print("   • Each platform optimized for different applications")

### 1.3 Decoherence Visualization

Let's visualize what T₁ and T₂ mean in practice.

In [ ]:
# Plot T1 and T2 decay for IBM backend
specs_ibm = get_backend_specs('ibm_jakarta')
avg_t1 = np.mean(specs_ibm.t1_times)
avg_t2 = np.mean(specs_ibm.t2_times)

fig = plot_decoherence_curves(t1=avg_t1, t2=avg_t2, max_time=300)
plt.show()

print(f"\nIBM Jakarta Decoherence:")
print(f"  T1 = {avg_t1:.1f} μs  (population decay)")
print(f"  T2 = {avg_t2:.1f} μs  (coherence decay)")
print(f"\nAt 1 μs:")
print(f"  T1 decay: {np.exp(-1/avg_t1):.4f} ({100*(1-np.exp(-1/avg_t1)):.2f}% lost)")
print(f"  T2 decay: {np.exp(-1/avg_t2):.4f} ({100*(1-np.exp(-1/avg_t2)):.2f}% lost)")
print(f"\nTypical gate times:")
print(f"  Single-qubit: ~50 ns  → {50/avg_t1*100:.3f}% T1 error")
print(f"  Two-qubit:   ~300 ns  → {300/avg_t1*100:.3f}% T1 error")
print(f"\n💡 This is why we can only do ~100 gates before noise dominates!")

---

## Part 2: Realistic Noise Modeling

Phase 4 taught us about noise *in theory*. Now let's see how noise behaves on *actual hardware*.

### 2.1 Creating Hardware-Specific Noise Models

In [ ]:
# Create noise models for each platform
print("REALISTIC NOISE MODEL CHARACTERIZATION")
print("="*70)

noise_models = {}
for backend_name in backends:
    noise_model = create_noise_model(backend_name)
    noise_models[backend_name] = noise_model
    
    summary = noise_model.characterization_summary()
    
    print(f"\n{backend_name.upper()}:")
    print("-"*70)
    print(f"  Single-qubit error: {summary['avg_single_qubit_error']:.6f}")
    print(f"  Two-qubit error:    {summary['avg_two_qubit_error']:.6f}")
    print(f"  Readout error:      {summary['avg_readout_error']:.6f}")
    print(f"  Avg T1: {summary['avg_t1_us']:.2f} μs")
    print(f"  Avg T2: {summary['avg_t2_us']:.2f} μs")

print("\n" + "="*70)

### 2.2 Circuit Fidelity vs Depth

One of the most important questions: **How deep can our circuits be before noise destroys the computation?**

In [ ]:
# Calculate fidelity for different circuit depths
depths = [10, 20, 50, 100, 200]

print("CIRCUIT FIDELITY vs DEPTH")
print("="*70)
print(f"\n{'Depth':<10} {'IBM':<15} {'IonQ':<15} {'Rigetti':<15}")
print("-"*70)

for depth in depths:
    # Assume roughly equal single and two-qubit gates
    num_sq = depth
    num_tq = depth // 2
    
    fidelities = []
    for backend_name in backends:
        fidelity = noise_models[backend_name].estimate_circuit_fidelity(num_sq, num_tq)
        fidelities.append(fidelity)
    
    print(f"{depth:<10} {fidelities[0]:<15.6f} {fidelities[1]:<15.6f} {fidelities[2]:<15.6f}")

print("\n" + "="*70)
print("\n💡 Key Insights:")
print("   • IBM: ~50% fidelity at depth 100")
print("   • IonQ: Better fidelity due to lower error rates")
print("   • Rigetti: Fidelity drops faster (higher 2Q error)")
print("   • All platforms: exponential decay F ≈ (1-p)^n")

In [ ]:
# Visualize fidelity vs depth
fig = plot_circuit_fidelity_vs_depth(backends, max_depth=100)
plt.show()

print("\n🔍 Analysis:")
print("   The exponential decay is why NISQ devices are limited to shallow circuits.")
print("   Error correction (Phase 5) will be needed for deep circuits!")

### 2.3 Running Phase 3 Algorithms with Realistic Noise

Remember Deutsch-Jozsa, Grover, and QFT from Phase 3? Let's see how they perform with realistic noise.

In [ ]:
# Simulate algorithms with different noise levels
from phase4_noise.density_matrix import DensityMatrix

print("PHASE 3 ALGORITHMS WITH REALISTIC NOISE")
print("="*70)

# Example: Bell state preparation
# Ideal: |Φ+⟩ = (|00⟩ + |11⟩)/√2  →  should measure 00 and 11 with 50% each

print("\nBell State Preparation: |Φ+⟩ = (|00⟩ + |11⟩)/√2")
print("-"*70)

# Create ideal Bell state
ideal_state = np.array([1, 0, 0, 1], dtype=complex) / np.sqrt(2)
rho_ideal = DensityMatrix(ideal_state)

print("\nIdeal (no noise):")
print(f"  Purity: {rho_ideal.purity():.6f}")
print(f"  Expected: 50% |00⟩, 50% |11⟩")

# Apply noise from each backend
for backend_name in backends:
    noise_model = noise_models[backend_name]
    
    # Simulate: H on qubit 0, CNOT(0,1)
    # Each gate introduces noise
    rho_noisy = DensityMatrix(ideal_state)  # Start fresh
    
    # H gate (single-qubit noise on qubit 0)
    rho_noisy = noise_model.apply_single_qubit_noise(rho_noisy, 0)
    
    # CNOT gate (two-qubit noise)
    rho_noisy = noise_model.apply_two_qubit_noise(rho_noisy, (0, 1))
    
    purity = rho_noisy.purity()
    
    print(f"\n{backend_name}:")
    print(f"  Purity: {purity:.6f} (ideal: 1.0)")
    print(f"  Purity loss: {(1-purity)*100:.2f}%")

print("\n" + "="*70)
print("\n💡 Even for a 2-gate circuit, noise causes measurable purity loss!")

---

## Part 3: Error Mitigation Techniques

In the NISQ era, we don't have enough qubits for error *correction* (Phase 5), but we can use error *mitigation* to improve results by 2-10x.

### 3.1 Readout Error Mitigation

Measurement errors are classical (bit flips), so we can characterize and invert them.

In [ ]:
print("READOUT ERROR MITIGATION")
print("="*70)

# Simulate readout errors for 2-qubit system
num_qubits = 2
readout_errors = [0.05, 0.03]  # 5% and 3% error rates

print(f"\nReadout error rates: {readout_errors}")
print(f"Expected outcomes: Bit flips during measurement")

# Create mitigator and calibrate
mitigator = ReadoutErrorMitigator(num_qubits)
mitigator.calibrate(readout_errors, num_shots=10000)

print(f"\n✓ Calibration complete")
print(f"  Calibration matrix shape: {mitigator.calibration_matrix.shape}")
print(f"  Condition number: {np.linalg.cond(mitigator.calibration_matrix):.2f}")

# Simulate noisy measurement counts
# True state: |00⟩ (should measure all 00)
ideal_counts = {'00': 1000, '01': 0, '10': 0, '11': 0}

# Add readout noise
np.random.seed(42)
noisy_counts = {'00': 850, '01': 80, '10': 40, '11': 30}

print(f"\nIdeal counts (|00⟩ state): {ideal_counts}")
print(f"Noisy counts (with readout errors): {noisy_counts}")

# Mitigate
mitigated_counts = mitigator.mitigate_counts(noisy_counts)

print(f"\nMitigated counts:")
for state in sorted(mitigated_counts.keys()):
    print(f"  |{state}⟩: {mitigated_counts[state]:.1f}")

print(f"\n✓ Mitigation recovered ~{mitigated_counts.get('00', 0)/1000*100:.0f}% accuracy!")
print(f"  (vs {noisy_counts['00']/1000*100:.0f}% without mitigation)")

### 3.2 Zero-Noise Extrapolation (ZNE)

**Key idea**: Run circuit at multiple noise levels, then extrapolate to zero noise!

Method:
1. Run circuit normally (noise level = 1x)
2. Run with artificially increased noise (3x, 5x, 7x)
3. Fit polynomial and extrapolate to noise = 0x

In [ ]:
print("ZERO-NOISE EXTRAPOLATION (ZNE)")
print("="*70)

# Simulate circuit execution at different noise levels
def noisy_circuit(fold_factor: int) -> float:
    """
    Simulate circuit with scaled noise.
    Fold factor: 1 = normal, 3 = triple noise, 5 = 5x noise, etc.
    """
    ideal_expectation = 1.0  # True expectation value
    base_noise = 0.05        # 5% error per gate
    
    # Error scales with fold factor
    noise_level = fold_factor * base_noise
    
    # Expectation decays with noise
    noisy_expectation = ideal_expectation * (1 - noise_level)
    
    # Add measurement noise
    noisy_expectation += np.random.normal(0, 0.01)
    
    return noisy_expectation

# Run ZNE
fold_factors = [1, 3, 5, 7]
print(f"\nFold factors: {fold_factors}")
print(f"(1 = normal noise, 3 = 3x noise, etc.)")

zne_result = ZeroNoiseExtrapolation.fold_circuit_globally(
    noisy_circuit,
    fold_factors,
    extrapolation_order=2
)

print(f"\n{zne_result}")

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))

# Measure at each fold factor
expectations = [noisy_circuit(f) for f in fold_factors]

# Plot data
ax.scatter(fold_factors, expectations, s=100, c='red', 
           label='Measured', zorder=5, edgecolor='black')

# Plot extrapolation
x_fit = np.linspace(0, max(fold_factors), 100)
coeffs = np.polyfit(fold_factors, expectations, 2)
y_fit = np.polyval(coeffs, x_fit)
ax.plot(x_fit, y_fit, 'b--', label='Polynomial fit', linewidth=2)

# Mark extrapolated value
ax.scatter([0], [zne_result.mitigated_expectation], s=200, c='green',
           marker='*', label='Extrapolated (zero noise)', zorder=5, edgecolor='black')

# Mark ideal
ax.axhline(y=1.0, color='green', linestyle=':', alpha=0.5, label='Ideal')

ax.set_xlabel('Noise Scale Factor', fontsize=12)
ax.set_ylabel('Expectation Value', fontsize=12)
ax.set_title('Zero-Noise Extrapolation', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

print(f"\n✓ ZNE improved result from {expectations[0]:.4f} to {zne_result.mitigated_expectation:.4f}")
print(f"  Ideal value: 1.0000")
print(f"  Error reduced by: {zne_result.improvement_factor:.2f}x")

### 3.3 Comparing Mitigation Techniques

In [ ]:
# Compare different mitigation methods
print("ERROR MITIGATION COMPARISON")
print("="*70)

ideal_value = 1.0
noisy_value = 0.70  # 30% error

mitigated_values = {
    'Readout': 0.85,   # ~2x improvement
    'ZNE': 0.92,       # ~3x improvement  
    'PEC': 0.96        # ~5x improvement (but high overhead)
}

print(f"\nIdeal expectation: {ideal_value:.4f}")
print(f"Noisy (unmitigated): {noisy_value:.4f} (error: {abs(noisy_value-ideal_value):.4f})")
print(f"\nMitigated results:")

for method, value in mitigated_values.items():
    error = abs(value - ideal_value)
    improvement = abs(noisy_value - ideal_value) / error
    print(f"  {method:<12} {value:.4f}  (error: {error:.4f}, {improvement:.1f}x better)")

# Visualize
fig = plot_error_mitigation_comparison(ideal_value, noisy_value, mitigated_values)
plt.show()

print("\n" + "="*70)
print("\n💡 Which mitigation to use?")
print("   • Readout: Always use! Low overhead, 2-3x improvement")
print("   • ZNE: For shallow circuits (<50 gates), moderate overhead")
print("   • PEC: When you need highest accuracy, high shot overhead")
print("   • Combine: Best results from using multiple techniques!")

---

## Part 4: Putting It All Together

Let's see the complete workflow: hardware selection → noise modeling → algorithm → mitigation

In [ ]:
print("COMPLETE NISQ WORKFLOW")
print("="*70)

print("\n1. SELECT HARDWARE")
print("-"*70)
chosen_backend = 'ibm_jakarta'
specs = get_backend_specs(chosen_backend)
print(f"   Platform: {chosen_backend}")
print(f"   Qubits: {specs.num_qubits}")
print(f"   2Q fidelity: {specs.gate_fidelities['cnot']:.4f}")

print("\n2. CREATE NOISE MODEL")
print("-"*70)
noise_model = create_noise_model(chosen_backend)
summary = noise_model.characterization_summary()
print(f"   Single-qubit error: {summary['avg_single_qubit_error']:.6f}")
print(f"   Two-qubit error: {summary['avg_two_qubit_error']:.6f}")

print("\n3. ESTIMATE CIRCUIT FIDELITY")
print("-"*70)
circuit_depth = 50
num_sq = circuit_depth
num_tq = circuit_depth // 2
fidelity = noise_model.estimate_circuit_fidelity(num_sq, num_tq)
print(f"   Circuit depth: {circuit_depth}")
print(f"   Gates: {num_sq} single-qubit + {num_tq} two-qubit")
print(f"   Expected fidelity: {fidelity:.4f} ({fidelity*100:.1f}%)")

print("\n4. DECISION: USE ERROR MITIGATION?")
print("-"*70)
if fidelity < 0.9:
    print(f"   ✓ YES - Fidelity {fidelity:.4f} is low, mitigation recommended")
    print(f"   Recommend: Readout + ZNE for 3-5x improvement")
else:
    print(f"   Fidelity {fidelity:.4f} is acceptable, mitigation optional")

print("\n5. ESTIMATED IMPROVEMENT")
print("-"*70)
mitigation_factor = 3.0  # Conservative estimate with ZNE
effective_fidelity = min(fidelity * mitigation_factor, 0.99)
print(f"   Without mitigation: {fidelity:.4f}")
print(f"   With mitigation (ZNE): ~{effective_fidelity:.4f}")
print(f"   Improvement: {mitigation_factor:.1f}x")

print("\n" + "="*70)
print("\n🎯 Result: Circuit is viable for NISQ hardware with error mitigation!")

---

## Summary & Key Takeaways

### What We Learned

1. **Hardware Platforms**
   - IBM: Balanced, good for learning and research
   - IonQ: Highest fidelity, all-to-all connectivity, but slow gates
   - Rigetti: Many qubits, but higher error rates
   - **No perfect platform** - choose based on application

2. **Realistic Noise**
   - Circuit fidelity decays **exponentially** with depth: F ≈ (1-p)^n
   - Current hardware limited to ~50-100 gates
   - T₁ and T₂ set fundamental limits

3. **Error Mitigation Works!**
   - **Readout calibration**: 2-3x improvement, always use
   - **ZNE**: 2-5x improvement, moderate overhead
   - **PEC**: 5-10x improvement, high overhead
   - **Combining techniques** gives best results

4. **NISQ is a Bridge**
   - Current era: Limited depth, need mitigation
   - Future: Error correction (Phase 5) enables deep circuits
   - Timeline: 5-15 years to fault-tolerant quantum computing

### Connection to Previous Phases

- **Phase 1-3**: Theory (how quantum computers *should* work)
- **Phase 4**: Challenges (why they're hard to build)
- **Phase 5**: Future solution (error correction)
- **Phase 6**: Present reality (NISQ + mitigation)

### Next Steps

Continue to **`06_phase6_advanced.ipynb`** to explore:
- Hardware benchmarking (Randomized Benchmarking, Quantum Volume)
- NISQ algorithms (VQE for chemistry, QAOA for optimization)
- Quantum teleportation protocol
- Career paths in quantum computing

---

### For Recruiters

This notebook demonstrates:
- ✅ Understanding of real quantum hardware (IBM, IonQ, Rigetti)
- ✅ Ability to model and analyze realistic noise
- ✅ Knowledge of state-of-the-art error mitigation techniques
- ✅ Complete NISQ workflow from hardware selection to execution
- ✅ Clear communication of complex technical concepts

**Ready to contribute to quantum computing companies from day one!**

---

*Author: Wadoud Charbak*  
*November 2024*  
*For: Quantinuum & Riverlane Recruitment*